Start by getting more data.

Include both snotel and weather station data.

In [248]:
import pandas as pd

def get_seasonal_nwac():
    crystal_summit_seasonal = pd.read_csv('../content/csv/seasonal_weather_crystal_summit.csv')
    # todo: aggregate from multiple sources
    return crystal_summit_seasonal

seasonal_nwac = get_seasonal_nwac()
seasonal_nwac.head()

,Date PDT,Temp F,RH %,Min mph,Spd mph,Gust mph,Dir deg
0,Apr 24 6:00,23,98,7,18,31,W
1,Apr 24 5:00,23,98,6,19,30,W
2,Apr 24 4:00,23,98,7,16,29,W
3,Apr 24 3:00,23,98,4,12,21,W
4,Apr 24 2:00,25,97,0,2,9,NW


In [249]:
seasonal_nwac.tail()

,Date PDT,Temp F,RH %,Min mph,Spd mph,Gust mph,Dir deg
5642,Sep 1 4:00,57,46,10,17,23,W
5643,Sep 1 3:00,57,43,9,16,23,W
5644,Sep 1 2:00,59,41,11,19,27,W
5645,Sep 1 1:00,59,37,8,17,23,W
5646,Sep 1 0:00,59,36,6,15,23,W


In [250]:
def get_seasonal_snotel():
    morse_lake_seasonal = pd.read_csv('../content/csv/seasonal_snotel_morse_lake.csv')
    # todo: aggregate from multiple sites
    return morse_lake_seasonal

seasonal_snotel = get_seasonal_snotel()
seasonal_snotel.head()

,Date PDT,Temp F,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
0,Apr 24 6:00,30.0,NaN,NaN,57.0,-0.5,126.0,1.0
1,Apr 24 5:00,30.0,NaN,NaN,57.1,-0.2,126.0,1.0
2,Apr 24 4:00,31.0,NaN,NaN,57.2,0.0,123.0,-2.0
3,Apr 24 3:00,31.0,NaN,NaN,57.2,0.2,122.0,-3.0
4,Apr 24 2:00,31.0,NaN,NaN,57.2,0.3,124.0,-2.0


In [251]:
seasonal_snotel.tail()

,Date PDT,Temp F,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
5593,Sep 1 5:00,53.0,86.3,0.0,NaN,NaN,0.0,0.0
5594,Sep 1 4:00,57.0,86.3,0.0,NaN,NaN,0.0,0.0
5595,Sep 1 3:00,62.0,86.3,0.0,NaN,NaN,0.0,0.0
5596,Sep 1 2:00,58.0,86.3,0.0,NaN,NaN,0.0,0.0
5597,Sep 1 1:00,58.0,86.3,0.0,0.0,0.0,0.0,0.0


todo: what happens if I drop columns I don't think are useful? How does that affect the prediciton accuracy? How does that align with my knowledge of snow science?

In [252]:
seasonal_nwac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5647 entries, 0 to 5646
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date PDT  5647 non-null   object
 1   Temp F    5647 non-null   int64 
 2   RH %      5647 non-null   int64 
 3   Min mph   5647 non-null   int64 
 4   Spd mph   5647 non-null   int64 
 5   Gust mph  5647 non-null   int64 
 6   Dir deg   5647 non-null   object
dtypes: int64(5), object(2)
memory usage: 308.9+ KB


In [253]:
seasonal_snotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date PDT   5598 non-null   object 
 1   Temp F     5597 non-null   float64
 2   PcpAc in   5068 non-null   float64
 3   ∆Pcp in    5047 non-null   float64
 4   SWE in     5290 non-null   float64
 5   ∆SWE  in   5266 non-null   float64
 6   SnoHt in   4800 non-null   float64
 7   ∆SnoHt in  4417 non-null   float64
dtypes: float64(7), object(1)
memory usage: 350.0+ KB


Join the dataframes (should have same date)

In [254]:
def get_seasonal_weather(nwac_df, snotel_df):
    seasonal_weather = nwac_df.join(snotel_df, lsuffix=" NWAC", rsuffix=" SNOTEL")
    seasonal_weather = seasonal_weather.drop('Date PDT SNOTEL', axis=1)
    seasonal_weather = seasonal_weather.rename(columns={'Date PDT NWAC': 'Date PDT'})
    return seasonal_weather

seasonal_weather = get_seasonal_weather(seasonal_nwac, seasonal_snotel)
seasonal_weather.head()

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
0,Apr 24 6:00,23,98,7,18,31,W,30.0,NaN,NaN,57.0,-0.5,126.0,1.0
1,Apr 24 5:00,23,98,6,19,30,W,30.0,NaN,NaN,57.1,-0.2,126.0,1.0
2,Apr 24 4:00,23,98,7,16,29,W,31.0,NaN,NaN,57.2,0.0,123.0,-2.0
3,Apr 24 3:00,23,98,4,12,21,W,31.0,NaN,NaN,57.2,0.2,122.0,-3.0
4,Apr 24 2:00,25,97,0,2,9,NW,31.0,NaN,NaN,57.2,0.3,124.0,-2.0


Read the danger ratings

In [255]:
def get_seasonal_danger():
    return pd.read_csv('../content/csv/danger_ratings.csv')

def remove_no_rating(danger_df):
    return danger_df[danger_df['Danger Rating'] != 'NO RATING']

seasonal_danger = get_seasonal_danger()
seasonal_danger = remove_no_rating(seasonal_danger)

def select_danger_area(danger_df, target_area='West Slopes South'):
    # only select target area
    return danger_df[(danger_df == target_area).any(axis=1)]

def map_dangerrating_to_number(danger_df):
    danger_number = { 'NO RATING': 0, 'LOW': 1, 'MODERATE': 2, 'CONSIDERABLE': 3, 'HIGH': 4, 'EXTREME': 5 }
    danger_df['Danger Rating'] = danger_df['Danger Rating'].map(danger_number)
    return danger_df

seasonal_danger = select_danger_area(seasonal_danger)
seasonal_danger = map_dangerrating_to_number(seasonal_danger)
seasonal_danger.head()

,Date,Area,Danger Rating
15,Apr 15 2023,West Slopes South,2
27,Apr 14 2023,West Slopes South,1
28,Apr 13 2023,West Slopes South,2
43,Apr 12 2023,West Slopes South,2
54,Apr 11 2023,West Slopes South,2


Wind direction -> number

In [256]:
def map_winddirection_to_number(weather_df):
    weather_number = { 
        'N': 0, 'NNE': 1, 'NE': 2, 'ENE': 3,
        'E': 4, 'ESE': 5, 'SE': 6, 'SSE': 7,
        'S': 8, 'SSW': 9, 'SW': 10, 'WSW': 11,
        'W': 12, 'WNW': 13, 'NW': 14, 'NNW': 15
    }
    weather_df['Dir deg'] = weather_df['Dir deg'].map(weather_number)
    return weather_df

seasonal_weather = map_winddirection_to_number(seasonal_weather)
seasonal_weather.head()

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
0,Apr 24 6:00,23,98,7,18,31,12,30.0,NaN,NaN,57.0,-0.5,126.0,1.0
1,Apr 24 5:00,23,98,6,19,30,12,30.0,NaN,NaN,57.1,-0.2,126.0,1.0
2,Apr 24 4:00,23,98,7,16,29,12,31.0,NaN,NaN,57.2,0.0,123.0,-2.0
3,Apr 24 3:00,23,98,4,12,21,12,31.0,NaN,NaN,57.2,0.2,122.0,-3.0
4,Apr 24 2:00,25,97,0,2,9,14,31.0,NaN,NaN,57.2,0.3,124.0,-2.0


Connect aggregated weather data to danger ratings

In [263]:
# todo: what happens if we change this?
# todo: how do we weight closer days higher?
# number of previous days to aggregate

        
import math

# --- Set things up
seasonal_nwac = get_seasonal_nwac()
seasonal_nwac = seasonal_nwac.fillna(0)
seasonal_nwac = map_winddirection_to_number(seasonal_nwac)

seasonal_snotel = get_seasonal_snotel()
seasonal_snotel = seasonal_snotel.fillna(0)

seasonal_weather = get_seasonal_weather(seasonal_nwac, seasonal_snotel)

seasonal_danger = get_seasonal_danger()
seasonal_danger = remove_no_rating(seasonal_danger)
seasonal_danger = select_danger_area(seasonal_danger)
seasonal_danger = map_dangerrating_to_number(seasonal_danger)
# --- Done setting things up


def setup_training_data(seasonal_df):
    training_data = pd.DataFrame([], columns=seasonal_df.columns)
    # Add columns for the new metrics
    training_data['Max Air Temp 24hr'] = None
    training_data['Max Air Temp 72hr'] = None
    training_data['Total Snowfall 24hr'] = None
    training_data['Total Snowfall 72hr'] = None
    training_data['Max Windspeed 24hr'] = None
    # Add column for danger rating
    training_data['Danger Rating'] = None
    return training_data

training_data = setup_training_data(seasonal_weather)

def connect_danger_to_aggregated_weather(seasonal_df, danger_df, result_df):
    def get_days_past(seasonal_df, days=3):
        return seasonal_df.iloc[idx+1:idx-1+24*days]
    
    def get_date_and_datestart(row):
        # Get the date (Month, day)
        date = row['Date'].split(' ')[0] + ' ' + row['Date'].split(' ')[1] # BAD
        # Get the time at the start of the day
        date_start = date + ' 0:00'
        return date, date_start
    
    def get_idx(seasonal_df, date_start):
        return seasonal_df[(seasonal_df == date_start).any(axis=1)].index[0]

    def aggregate(days_df):
        def get_past_day(days_df):
            # seems to work correctly
            return days_df.iloc[0:24]
        def get_max_air_temp(day):
            return day['Temp F NWAC'].max()
        def get_total_snofall(day):
            return day['PcpAc in'].sum()
        def get_max_wind(day):
            return day['Spd mph'].max() # should we use gust?

        past_day = get_past_day(days_df)
        max_temp_24 = get_max_air_temp(past_day)
        max_temp_72 = get_max_air_temp(days_df)
        snowfall_24 = get_total_snofall(past_day)
        snowfall_72 = get_total_snofall(days_df)
        max_wind_24 = get_max_wind(past_day)
        days_df = days_df.drop('Date PDT', axis=1).mean()
        days_df['Max Air Temp 24hr'] = max_temp_24
        days_df['Max Air Temp 72hr'] = max_temp_72
        days_df['Total Snowfall 24hr'] = snowfall_24
        days_df['Total Snowfall 72hr'] = snowfall_72
        days_df['Max Windspeed 24hr'] = max_wind_24

        return days_df
    
    def add_and_reindex(result_df, row):
        result_df.loc[-1] = row
        result_df.index = result_df.index+1
        return result_df
    
    def add_date_and_danger(row, date, danger):
        row['Date PDT'] = date
        row ['Danger Rating'] = danger
        return row
    
    # For every danger rating (ie for every day)
    for idx, row in danger_df.iterrows():
        date, date_start = get_date_and_datestart(row)
        # Use this to index the seasonal dataframe
        idx = get_idx(seasonal_df, date_start)
        # Get the past x days
        days_past = get_days_past(seasonal_df)
        # Aggregate them
        agg_past = aggregate(days_past)
        # Tag with date and danger rating
        agg_past = add_date_and_danger(agg_past, date, row['Danger Rating'])
        # Add back into the dataframe
        result_df = add_and_reindex(result_df, agg_past)
    # Drop a BUNCH of columns
    cols = [1,2,3,4,5,6,7,8,9,10,11,12,13] # BAD
    result_df.drop(result_df.columns[cols], axis=1, inplace=True)
    return result_df

training_data = connect_danger_to_aggregated_weather(seasonal_weather, seasonal_danger, training_data)
training_data.iloc[110:120]

,Date PDT,Max Air Temp 24hr,Max Air Temp 72hr,Total Snowfall 24hr,Total Snowfall 72hr,Max Windspeed 24hr,Danger Rating
35,Dec 28,35.0,40.0,644.6,1776.1,56.0,2
34,Dec 27,37.0,40.0,594.6,1689.9,43.0,3
33,Dec 26,40.0,40.0,584.0,1626.2,35.0,3
32,Dec 25,35.0,35.0,556.6,1572.6,39.0,3
31,Dec 24,32.0,32.0,529.6,1546.8,34.0,3
30,Dec 23,27.0,27.0,530.6,1533.1,36.0,4
29,Dec 22,8.0,25.0,530.2,1511.7,37.0,3
28,Dec 21,25.0,25.0,514.9,1488.6,37.0,3
27,Dec 20,21.0,31.0,508.8,1480.2,16.0,3
26,Dec 20,21.0,31.0,508.8,1480.2,16.0,3


Clean up NaN's

Replacing with 0 for now, but todo: is there a better way?

In [265]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor # used regressor originally

# date isn't a number, and isn't relevant (yet)
try:
    training_data = training_data.drop('Date PDT', axis=1)
except:
    pass

drop = 'Danger Rating'
x = training_data.drop(drop, axis=1)
y = training_data[drop]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

rf_regressor = RandomForestRegressor(random_state=0)

rf_regressor.fit(x_train, y_train)

y_pred_train = rf_regressor.predict(x_train)
r2_score(y_train, y_pred_train)


0.8401776422764228

In [266]:
y_pred = rf_regressor.predict(x_test)
r2_score(y_test, y_pred)

-0.026367768595041197

Wow...it got even worse. ~yay~